## Python Bibliotheken laden
Falls einzelne Bibliotheken noch nicht installiert wurden, muss das erst, beispielsweise mit %pip gemacht werden (z.B. %pip install pandas)

In [ ]:
#%pip install prophet

In [1]:
import pandas as pd
import requests
from io import StringIO

/Users/derungcu/Library/Python/3.10/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


## Velozählungen lesen und zusammenführen

Die Velozähldaten werden direkt von OpenData Zürich gelesen. Jedes Jahr liegt als eigenes CSV vor und wird als Pandas Dataframe geladen.

In [10]:
# Funktion um CSV mit Velozähldaten für ein Jahr von URL zu lesen
def get_velo_data(year):
    url = f"https://data.stadt-zuerich.ch/dataset/ted_taz_verkehrszaehlungen_werte_fussgaenger_velo/download/{year}_verkehrszaehlungen_werte_fussgaenger_velo.csv"
    response = requests.get(url)
    response.raise_for_status()  # This will raise an error if the download failed
    csv_raw = StringIO(response.text)
    df = pd.read_csv(csv_raw)
    return df

In [11]:
# CSV lesen
velo_23 = get_velo_data("2023")
velo_22 = get_velo_data("2022")
velo_21 = get_velo_data("2021")

# Daten kombiniere
velo = pd.concat([velo_21, velo_22, velo_23])

## Standort auswählen

In [12]:
velo = velo[velo['FK_STANDORT'] == 2989]

## Zeitfeld definieren

In [13]:
velo['DATUM'] = pd.to_datetime(velo['DATUM'], format="%Y-%m-%dT%H:%M")

## Stundenaggregation

In [18]:
# Aggregating values per hour
velo['DATUM_hour'] = velo['DATUM'].dt.floor('H')
velo_hour = velo.groupby('DATUM_hour').agg({'VELO_IN': 'sum', 'VELO_OUT': 'sum'})
velo_hour['VELO'] = velo_hour['VELO_IN'] + velo_hour['VELO_OUT']

## Daten für Prophet vorbereiten

In [20]:
# Renaming columns for Prophet
velo_proph = velo_hour.reset_index().rename(columns={'DATUM_hour': 'ds', 'VELO': 'y'}).drop(["VELO_IN", "VELO_OUT"], axis=1)

In [23]:
velo_proph.head(10)

,ds,y
0,2021-01-01 00:00:00,22.0
1,2021-01-01 01:00:00,53.0
2,2021-01-01 02:00:00,73.0
3,2021-01-01 03:00:00,53.0
4,2021-01-01 04:00:00,36.0
5,2021-01-01 05:00:00,20.0
6,2021-01-01 06:00:00,23.0
7,2021-01-01 07:00:00,17.0
8,2021-01-01 08:00:00,10.0
9,2021-01-01 09:00:00,20.0


## Daten speichern

In [24]:
velo_proph.to_csv("../Daten/velozaehldaten_aufbereitet.csv")